<a href="https://colab.research.google.com/github/rtmonteiro/ia-2023-1/blob/main/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
def repo(scores, classifier=None):
    from scipy import stats
    import numpy as np
    if classifier:
      print(f"{classifier}: ")
    print(f'Media: {scores.mean():.2f}, Desvio Padrao: {scores.std():.2f}')
    inf, sup = stats.norm.interval(0.95, loc=scores.mean(), 
                               scale=scores.std()/np.sqrt(len(scores)))
    print(f'Intervalo de confiança (95%): [{inf:.2f},{sup:.2f}]\n')

1. Compare a acurácia da árvore de decisão que utiliza ganho de informação com aquela que usa
índice gini para seleção da característica dos nós de decisão da árvore no dataset wine. Faça a
comparação usando 6 rodadas de validação cruzada estratificada com 5 folds. A menos do critério
de seleção de caraterísticas, use os valores default para os demais hiperparâmetros da árvore.
Indique se existe diferença significativa entre os resultados das árvores usando o teste t de Student.

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_wine
from scipy import stats

wine = load_wine()
wine_X = wine.data
wine_Y = wine.target

#Ganho de informação
dt_ent = DecisionTreeClassifier(criterion = 'entropy')

rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=6)

scores1 = cross_val_score(dt_ent, wine_X, wine_Y, cv = rkf)

#Gini
dt_gini = DecisionTreeClassifier()

rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=6)

scores2 = cross_val_score(dt_gini, wine_X, wine_Y, 
                         cv = rkf)

stats.ttest_ind(scores1, scores2)

Ttest_indResult(statistic=1.771705066201773, pvalue=0.0816965987499078)

2. Determine qual o valor do hiperparâmetro ccp_alpha (fator de poda) em uma busca em grade
com validação cruzada em 10 folds no dataset wine que obtém a melhor acurácia média. Varie o
hiperparâmetro de 0.1 em 0.1 no intervalo entre 0.1 e 0.7.

In [12]:
from sklearn.model_selection import GridSearchCV

parameters = {'ccp_alpha': [i/10 for i in range(1,8)]}

dt = DecisionTreeClassifier()

clf = GridSearchCV(dt, parameters,cv=10)

clf.fit(wine_X,wine_Y)

print(clf.best_params_)
print(clf.best_score_)

{'ccp_alpha': 0.1}
0.8045751633986928


3. Compare o desempenho em f1 macro do classificador Naive Bayes com os do classificadores
Árvore de Decisão (com valores default de hiperparâmetros) e com o classificador aleatório
estratificado em uma validação cruzada com 10 folds no dataset breast.

In [25]:
from sklearn.datasets import load_breast_cancer
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier

breast = load_breast_cancer()
breast_X = breast.data
breast_y = breast.target

# Naive Bayes
gNB = GaussianNB()

scores_gnb = cross_val_score(gNB, breast_X, breast_y, cv=10, scoring="f1")

repo(scores_gnb, "Naive Bayes")

# Decision Tree
dt = DecisionTreeClassifier()

scores_dt = cross_val_score(dt, breast_X, breast_y, cv=10, scoring="f1")
repo(scores_dt, "Decision Tree")

# Aleatorio estratificado
dc = DummyClassifier(strategy="stratified")

scores_dummy = cross_val_score(dc, breast_X, breast_y, cv=10, scoring="f1")

repo(scores_dummy, "Dummy Classifier")


Naive Bayes: 
Media: 0.95, Desvio Padrao: 0.02
Intervalo de confiança (95%): [0.94,0.96]

Decision Tree: 
Media: 0.93, Desvio Padrao: 0.02
Intervalo de confiança (95%): [0.92,0.94]

Dummy Classifier: 
Media: 0.60, Desvio Padrao: 0.06
Intervalo de confiança (95%): [0.56,0.63]



4. Obtenha a acurácia média, o desvio padrão e o intervalo de confiança a 95% do classificador
Perceptron de Múltiplas Camadas usando validação cruzada com 10 dobras (folds) na base de dados
(dataset) wine padronizada e não padronizada. Altere manualmente o valor da taxa de aprendizado
inicial no melhor classificador para 0.1, 0.01 e 0.0001 e observe o resultado.

In [40]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

repo(cross_val_score(MLPClassifier(activation="identity"), wine_X, wine_Y, cv=10), "MLP")

pipe = Pipeline([('scaler', StandardScaler()), ('estimator', MLPClassifier(activation="identity"))])

repo(cross_val_score(pipe, wine_X, wine_Y, cv=10), "MLP + StandardScaler")

for j in [0.1, 0.01, 0.0001]:
  pipe = Pipeline([('scaler', StandardScaler()), ('estimator', MLPClassifier(learning_rate_init=j, activation="identity"))])

  repo(cross_val_score(pipe, wine_X, wine_Y, cv=10), f"MLP(learning rate:{j}) + StandardScaler")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

MLP: 
Media: 0.70, Desvio Padrao: 0.17
Intervalo de confiança (95%): [0.60,0.81]



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

MLP + StandardScaler: 
Media: 0.98, Desvio Padrao: 0.03
Intervalo de confiança (95%): [0.96,0.99]

MLP(learning rate:0.1) + StandardScaler: 
Media: 0.98, Desvio Padrao: 0.03
Intervalo de confiança (95%): [0.96,0.99]

MLP(learning rate:0.01) + StandardScaler: 
Media: 0.98, Desvio Padrao: 0.03
Intervalo de confiança (95%): [0.97,1.00]



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

MLP(learning rate:0.0001) + StandardScaler: 
Media: 0.94, Desvio Padrao: 0.06
Intervalo de confiança (95%): [0.91,0.98]



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
